In [205]:
## from sklearn.ensemble import RandomForestClassifier
import pandas as pd
X1 = pd.read_csv("dataset.csv", usecols=range(1,16))
Y1 = pd.read_csv("dataset.csv", usecols = [16])
X = X1.values
Y = Y1.values
y = [x[0] for x in Y]
ytrain = y[0:24000]
ycheck = y[24000:35898]
Train = X[0:24000]
Check = X[24000:35898]
Rf = RandomForestClassifier(n_estimators = 1, criterion="entropy", max_features = None, bootstrap = False)
Rf.fit(Train,ytrain)
Exits = Rf.score(Check,ycheck)
print("%Exits amb tots els atributs =", Exits)
"""for i in range(0,15):
    Traini = Train.drop(Train.columns[i], axis = 1)
    Rf.fit(Traini,ytrain)
    Checki = Check.drop(Check.columns[i], axis = 1)
    Exits = Rf.score(Checki,ycheck)
    print("%Exits sense ", Train.columns[i], " =", Exits)"""

%Exits amb tots els atributs = 0.822407127248277


'for i in range(0,15):\n    Traini = Train.drop(Train.columns[i], axis = 1)\n    Rf.fit(Traini,ytrain)\n    Checki = Check.drop(Check.columns[i], axis = 1)\n    Exits = Rf.score(Checki,ycheck)\n    print("%Exits sense ", Train.columns[i], " =", Exits)'